In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
sales = pd.read_csv('../input/seattle-house-sales-prices/house_sales.csv')
sales



In [ ]:
sales.count()

In [ ]:
pd.value_counts(sales['bedrooms'])

Here we want to find out which houses have had the most visits or views. This variable is important to find out what are the factors that make a home have many (or few) visits.

In [ ]:
sales['view']
sales.sort_values(by='view', ascending=False)

Now we will filter with more variables to find out if we have real estate available according to our search criteria.

In [ ]:
sales.loc[(sales['bedrooms']==3) & (sales['bathrooms'] > 2) & (sales['price'] < 200000) & (sales['yr_built']> 1990)]

Great Job! We found homes according to our search criteria. This is very exciting! 

And now, let's start the create plot of graphics. Here, we start our analyses explorate. 


In [ ]:
sales.plot(x='bathrooms',y='price',kind='scatter',color='y')

In [ ]:
sns.lmplot(x='bathrooms',y='price',data=sales)

Ok. We have correlations posite between bathrooms and price. 


In [ ]:
sns.lmplot(x='sqft_living',y='price',data=sales)

In [ ]:
sns.lmplot(x='bedrooms',y='price',data=sales)

In [ ]:
sns.lmplot(x='view',y='price',data=sales)

We can confirm the correlations found in the graphs above by correlating with all variables.


In [ ]:
sns.heatmap(sales.corr(),cmap='coolwarm',annot=True)
plt.rcParams['figure.figsize'] = (15.5, 10.5)

Now let's start regression analysis to identify how much each of these variables interferes with real estate prices. I will run more than one regression model. I want to extract the best possible model.


**Training a Regression Linear Model**

Let's now start training the regression model. We first need to split our data into an X matrix that contains the resources to train, and a y matrix with the target variable, in this case, the Price column. Let's discard the "Adress" column because it only has textual information than the regression model. Linear can not use.

In [ ]:
X = sales[['sqft_living','floors','waterfront','grade','sqft_lot', 'bedrooms', 'bathrooms', 'view']]
y = sales['price']

**Split Training**


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

**CREATE AND TRAINING THE MODEL**

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
lm.score(X_train, y_train)

Model Evaluation

In [ ]:
print(lm.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

In [ ]:
predictions = lm.predict(X_test)
print(y_test,predictions)

In [ ]:
plt.scatter(y_test,predictions)

The more expensive the property is, the less successful the model can get.

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
plt.figure(figsize=(25,8))

plt.plot(range(0, y_test.shape[0]), y_test, marker='+')
plt.plot(range(0, predictions.shape[0]), predictions, marker='o')

This chart confirms our evidence that the model is not performing for higher priced real estate. We would have to try other approaches, other predictive models to improve the performance of our model.

In [ ]:
import folium
from folium import plugins

In [ ]:
lat = sales['lat'][:500].values
long = sales['long'][:500].values

mapa = folium.Map(location=[47.608013,-122.335167],zoom_start=4)

for la,lo in zip(lat,long):
    folium.Marker([la, lo]).add_to(mapa)

mapa

In [ ]:
coordenadas = []
lat = sales['lat'][:500].values
long = sales['long'][:500].values

mapa = folium.Map(location=[47.608013,-122.335167],zoom_start=4)

for la,lo in zip(lat,long):
    coordenadas.append([la,lo])

mapa.add_child(plugins.HeatMap(coordenadas))
mapa